Week 5 - Transport layer pt 2

Building a simple tcp server

In [ ]:
import socket 
 
# Create a TCP socket 
server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM) 
server_socket.bind(('localhost', 65432)) 
server_socket.listen(1)  # Allow 1 pending connection 
 
print("TCP Server is listening...") 
 
while True: 
    client_socket, client_address = server_socket.accept() 
    print(f"Connected to {client_address}") 
     
    data = client_socket.recv(1024) 
    print(f"Received: {data.decode()}") 
     
    # Echo back the data 
    client_socket.sendall(b"ACK: " + data) 
    client_socket.close()

In [ ]:
import socket 
 
client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM) 
client_socket.connect(('localhost', 65432)) 
 
message = input("Enter message: ") 
client_socket.sendall(message.encode()) 
 
response = client_socket.recv(1024) 
print(f"Server response: {response.decode()}") 
 
client_socket.close()

Exercise 1

In [ ]:
import socket 
import datetime
 
client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM) 
client_socket.connect(('localhost', 65432)) 

message = input("Enter message: ") 

start_time = datetime.datetime.now()

client_socket.sendall(message.encode()) 

response = client_socket.recv(1024) 


end_time = datetime.datetime.now()

trans_time = end_time - start_time

print(f"Server response: {response.decode()}") 
print(f"Time: {trans_time}") 
client_socket.close()

Exercise 2

In [ ]:
import socket

server_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
server_socket.bind(('localhost', 65433))

print("UDP server is ready to receive API data...")

while True:
        data, client_address = server_socket.recvfrom(2048)
        print(f"Received data from {client_address}: {data.decode()}")

In [ ]:
import socket 
import datetime

client_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM) 
client_socket.connect(('localhost', 65433)) 
 
message = input("Enter message: ") 

start_time = datetime.datetime.now()

client_socket.sendall(message.encode()) 
 
end_time = datetime.datetime.now()
trans_time = end_time - start_time

print(f"Time: {trans_time}") 

client_socket.close()

Exercise 3

In [ ]:
import socket

server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server_socket.bind(('localhost', 65432))
server_socket.listen(1) 

print("TCP Server is listening...")

while True:
    client_socket, client_address = server_socket.accept()
    print(f"Connected to {client_address}")

    # Receive the file and save it
    with open('received_file.txt', 'wb') as f:
        while True:
            data = client_socket.recv(1024)
            if not data:
                break
            f.write(data)
    print("File received")

    # Send an acknowledgment back to the client
    client_socket.sendall(b"ACK: File received")
    client_socket.close()

In [ ]:
import socket
import datetime

client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
client_socket.connect(('localhost', 65432))

# Record the timestamp before sending the file
start_time = datetime.datetime.now()

# Read and send the file
with open('file_to_send.txt', 'rb') as f:
    client_socket.sendfile(f)

# Receive the server's response
response = client_socket.recv(1024)

# Record the timestamp after receiving the response
end_time = datetime.datetime.now()

# Calculate the transmission time
trans_time = end_time - start_time

print(f"Server response: {response.decode()}")
print(f"Time: {trans_time.total_seconds()} seconds")

client_socket.close()

Exercise 4

In [ ]:
import socket
import datetime

client_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
server_address = ('localhost', 65433)
start_time = datetime.datetime.now()

with open('file_to_send.txt', 'rb') as f:
    chunk_num = 0
    while True:
        chunk = f.read(1024)
        if not chunk:
            break
        message = f"{chunk_num}".encode() + b":" + chunk
        client_socket.sendto(message, server_address)
        
        ack, _ = client_socket.recvfrom(1024)
        if ack.decode() != f"ACK:{chunk_num}":
            print(f"Failed to receive ACK {chunk_num}")
            break
        chunk_num += 1

client_socket.sendto(b"EOF", server_address)

end_time = datetime.datetime.now()

trans_time = end_time - start_time

print(f"Time: {trans_time.total_seconds()} seconds")

client_socket.close()

In [ ]:
import socket

server_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
server_socket.bind(('localhost', 65433))

print("UDP server is ready to receive file data...")

received_chunks = {}

while True:
    data, client_address = server_socket.recvfrom(2048)
    
    if data == b"EOF":
        print("File transfer complete")
        break
    
    chunk_num, chunk_data = data.split(b":", 1)
    chunk_num = int(chunk_num.decode())
    
    received_chunks[chunk_num] = chunk_data
    
    ack_message = f"ACK:{chunk_num}".encode()
    server_socket.sendto(ack_message, client_address)

with open('receivedfile.txt', 'wb') as f:
    for chunk_num in sorted(received_chunks.keys()):
        f.write(received_chunks[chunk_num])

print("File received and saved as 'receivedfile.txt'")

Exercise 5

In [ ]:

import socket
import threading

def handle_client(client_socket, client_address):
    print(f"Connected to {client_address}")
    
    with open(f'receivedfile_{client_address[1]}.txt', 'wb') as f:
        while True:
            data = client_socket.recv(1024)
            if not data:
                break
            f.write(data)
    print(f"File received from {client_address}")

    client_socket.sendall(b"ACK: File received")
    client_socket.close()

server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server_socket.bind(('localhost', 65432))
server_socket.listen(5)  

print("TCP Server is waiting")

while True:
    client_socket, client_address = server_socket.accept()
    client_handler = threading.Thread(target=handle_client, args=(client_socket, client_address))
    client_handler.start()

Exercise 6 + 7

In [ ]:
import socket
import threading
from cryptography.fernet import Fernet as fn

key = fn.generate_key()
cipher_suite = fn(key)
print(f"Key: {key}") 

def client_manager(client_socket, client_address, second_client_socket):
    print(f"Connected to {client_address}")

    while True:
        try:
            data = client_socket.recv(2048)
            if not data:
                break
            decrypted_data = cipher_suite.decrypt(data).decode()
            print(f"Received data from {client_address}: {decrypted_data}")


server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server_socket.bind(('localhost', 65432))
server_socket.listen(2) 

print("TCP Server is waitint")

client_socket1, client_address1 = server_socket.accept()
client_socket2, client_address2 = server_socket.accept()

thread1 = threading.Thread(target=handle_client, args=(client_socket1, client_address1, client_socket2))
thread2 = threading.Thread(target=handle_client, args=(client_socket2, client_address2, client_socket1))
thread1.start()
thread2.start()

thread1.join()
thread2.join()
server_socket.close()

In [ ]:

import socket
import threading
from cryptography.fernet import Fernet

key = b'your_saved_key'  
cipher_suite = Fernet(key)

def receive_messages(client_socket):
    while True:
        try:
            data = client_socket.recv(2048)
            decrypted_message = cipher_suite.decrypt(data).decode()
            print(f"Received message: {decrypted_message}")
        except:
            break

client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
client_socket.connect(('localhost', 65432))

# Start a thread to listen for incoming messages
receive_thread = threading.Thread(target=receive_messages, args=(client_socket,))
receive_thread.daemon = True
receive_thread.start()

while True:
    message = input("Enter message or type `disconnect` to close: ")
    encrypted_message = cipher_suite.encrypt(message.encode())
    
    # Sending the encrypted message to the server
    client_socket.sendall(encrypted_message)
    
    if message == "disconnect":
        print("Disconnecting")
        client_socket.close()
        break

Exercise 8

In [ ]:
import requests
import socket
import json

api_url = "https://api.open-meteo.com/v1/forecast?latitude=51.47&longitude=-0.0363&current_weather=true"

response = requests.get(api_url)

if response.status_code == 200:
    data = response.json()  
    current_weather = data['current_weather']   
    temperature = current_weather['temperature']  
else:
    print("Failed to get weather data")        


server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server_socket.bind(('localhost', 65432))
server_socket.listen(1) 

print("TCP Server is waiting")
client_socket, client_address = server_socket.accept()
print(f"Connected to {client_address}") 
if client_socket:
    client_socket.sendall(f"Temperature: {temperature}".encode())
    client_socket.sendall(f"Current weather: {current_weather}".encode())
    client_socket.close()
